In [1]:
import string
from datetime import date, datetime
from math import ceil
import re
import shutil
from pathlib import Path

In [2]:
def daily_words_calculate(word_goal, goal_start_date, goal_finish_date):
    # We currently assume you are starting at zero words, or are factoring your already existant words into your decision.
    # This assumption may be worth revisiting later
    #NOTE: changed incoming date format #days_left = abs((datetime.strptime(goal_finish_date,"%d/%m/%Y") - datetime.strptime(goal_start_date,"%d/%m/%Y")).days)
    days_left = abs((datetime.strptime(goal_finish_date,"%Y-%m-%d") - datetime.strptime(goal_start_date,"%Y-%m-%d")).days)
    daily_target = ceil(int(word_goal)/days_left)
    return daily_target


def word_goal_calculate(daily_target, goal_start_date, goal_finish_date):
    # We currently assume you are starting at zero words, or are factoring your already existant words into your decision.
    # This assumption may be worth revisiting later
    #NOTE: changed incoming date format #days_left = abs((datetime.strptime(goal_finish_date,"%d/%m/%Y") - datetime.strptime(goal_start_date,"%d/%m/%Y")).days)
    days_left = abs((datetime.strptime(goal_finish_date,"%Y-%m-%d") - datetime.strptime(goal_start_date,"%Y-%m-%d")).days)
    word_goal = int(daily_target)*days_left
    return word_goal

In [3]:
def change_goal(goal_start_date, goal_end_date, word_goal=None, daily_target=None):
    if (word_goal is not None) and (daily_target is None):
        goal = daily_words_calculate(word_goal, goal_start_date, goal_end_date)
        return goal
    elif (daily_target is not None) and (word_goal is None):
        goal = word_goal_calculate(daily_target, goal_start_date, goal_end_date)
        return goal
    else:
        raise Exception("Error: provide word_goal or daily_target, but not both.")

In [4]:
import sqlite3
sqlite3_path = './database/nanite_storage.sqlite3'

query = '''INSERT INTO words values (
    ?, ?, ?, ?, ?, ?) '''
params = [None, 1, 1, '2021-03-12 08:00:00.000000', 22000, 1000]
conn = sqlite3.connect(sqlite3_path)
cur = conn.cursor()
cur.execute(query, params)
conn.commit()

In [5]:
from collections import namedtuple
WordVals = namedtuple('Wordcounts', 'record_id project_id author_id Wdate Wcount Wtarget')

In [6]:
import pandas as pd


In [75]:
def get_words_time(freq=None) -> dict:
    freq = freq
    conn = sqlite3.connect(sqlite3_path)
    cur = conn.cursor()
    cur.execute("SELECT * FROM words where project_id=?", '1')
    row = cur.fetchall()
    data = [dict(WordVals(*row[i])._asdict()) for i in range(len(row))]
    conn.close()
    df = pd.DataFrame(data)
    df.index = pd.DatetimeIndex(df['Wdate'])

    earliest = df.index[0]
    latest = df.index[-1]

    new_index = pd.date_range(earliest, latest, freq='D')

    df = df.groupby(pd.Grouper(freq='D')).last()
    df = df.reindex(new_index, method='ffill')
    df = df.fillna(method='ffill')
    df['Wtarget_sum'] = df['Wtarget'].cumsum()
    df['Wdate'] = df['Wdate'].apply(lambda x: x[:10])
    df = df.groupby(pd.Grouper(freq=freq)).last()
    records = df.to_dict(orient='records')
    return records

[{'Wcount': 1500.0,
  'Wdate': '2021-03-01',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 1.0,
  'Wtarget_sum': 1000.0},
 {'Wcount': 2300.0,
  'Wdate': '2021-03-02',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 2.0,
  'Wtarget_sum': 2000.0},
 {'Wcount': 2900.0,
  'Wdate': '2021-03-03',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 3.0,
  'Wtarget_sum': 3000.0},
 {'Wcount': 3500.0,
  'Wdate': '2021-03-04',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 4.0,
  'Wtarget_sum': 4000.0},
 {'Wcount': 3500.0,
  'Wdate': '2021-03-04',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 4.0,
  'Wtarget_sum': 5000.0},
 {'Wcount': 5200.0,
  'Wdate': '2021-03-06',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 5.0,
  'Wtarget_sum': 6000.0},
 {'Wcount': 6500.0,
  'Wdate': '2021-03-07',
  'Wtarget': 1000.0,
  'author_id': 1